# **NLP1_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """ A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:

- Profit functions: The profit earned per unit of 1.5l Coli and Pepsa products are given as 49000X1 - X1^2 and 30X2 - 2*X2^2 respectively.
- Cost of production: It costs $150 to buy and process each hectolitre of syrup. 
- Production capacity: The S1 packaging line can produce up to 7100 units of product per hour. Calculating it for 5 days with 8 hours shift, it is 7100 * 8 * 5 = 2,84,000 units per week.
- Production ratio: The firm must produce at least twice the amount of COLI than PEPSA i.e., X1 >= 2*X2.

Decision Variables:

- X1: Number of 1.5l COLI units produced
- X2: Number of 1.5l PEPSA units produced

Objective Function:

Maximize Z = [49000*X1 - X1^2 + 30*X2 - 2*X2^2] - 150*(X1/40 + X2/20) 
          
(Z is the net profit in dollars from the sale of COLI and PEPSA minus the cost of syrup.)

Subject to the following constraints:

1. Production capacity constraint: X1 + X2 <= 2,84,000 (Assuming that the total weekly production capacity can be equally divided among

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response2.choices[0].message.content)

Below is the Pyomo code for the stated optimization problem. Sample data has been used wherever the actual data values are missing in the problem statement.

```python
from pyomo.environ import *

# Concrete Model
model = ConcreteModel()

# Decision Variables
model.X1 = Var(domain=NonNegativeReals) # Number of 1.5l COLI units produced
model.X2 = Var(domain=NonNegativeReals) # Number of 1.5l PEPSA units produced

# Objective function
model.Profit = Objective(expr=(49000*model.X1 - model.X1**2 + 30*model.X2 - 2*model.X2**2) - 150*(model.X1/40 + model.X2/20),
                         sense=maximize)

# Constraints
model.Capacity_Constraint = Constraint(expr=model.X1 + model.X2 <= 284000)
model.Balance_Constraint = Constraint(expr=model.X1 - 2*model.X2 >= 0)

# solver
solver = SolverFactory('glpk')

# solve
solver.solve(model)

# Print the output
print("Number of COLI units =", model.X1(), "Number of PEPSA units =", model.X2())
```

If Pyomo and GLPK solver are correctly installed, you should be able to use the script above to find the number of COLI and PEPSA units that maximize the net profit. You can replace the solver 'glpk' with any other solver installed in your environment.

### **Run the code Generated by GPT4**

In [ ]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

In [ ]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [ ]:
from pyomo.environ import *

# Concrete Model
model = ConcreteModel()

# Decision Variables
model.X1 = Var(domain=NonNegativeReals) # Number of 1.5l COLI units produced
model.X2 = Var(domain=NonNegativeReals) # Number of 1.5l PEPSA units produced

# Objective function
model.Profit = Objective(expr=(49000*model.X1 - model.X1**2 + 30*model.X2 - 2*model.X2**2) - 150*(model.X1/40 + model.X2/20),
                         sense=maximize)

# Constraints
model.Capacity_Constraint = Constraint(expr=model.X1 + model.X2 <= 284000)
model.Balance_Constraint = Constraint(expr=model.X1 - 2*model.X2 >= 0)

# solver
solver = SolverFactory('knitro')

# solve
solver.solve(model)

# Print the output
print("Number of COLI units =", model.X1(), "Number of PEPSA units =", model.X2())
print(model.Profit())

Number of COLI units = 24498.125 Number of PEPSA units = 5.62500000126044
600158191.796875


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**